<a href="https://colab.research.google.com/github/jiyoon0227/fashion-customer-churn-analysis/blob/main/01_data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **비즈니스 문제 정의**
- 패션 이커머스는 반복 구매가 핵심
- 고객 이탈은 매출 감소로 직결
- 이탈 위험 고객을 조기 탐지해야 함

In [1]:
import pandas as pd
# 거래 데이터 불러오기
transactions=pd.read_parquet("./data/transactions_train.parquet")

In [2]:
transactions.shape

(31788324, 5)

In [5]:
# 날짜 타입 변환 ( 날짜 계산하려면 datetime 타입 )
transactions['t_dat']=pd.to_datetime(transactions['t_dat'])

In [7]:
# 데이터 중에서 가장 최근 구매일
latest_date=transactions['t_dat'].max()
latest_date

Timestamp('2020-09-22 00:00:00')

## **RFM**
### 고객 행동을 요약하는 기본적인 프레임워크 지정
- Recency : 마지막 구매 이후 경과일
- Frequency : 구매 횟수
- Monetary : 총 구매 금액

In [8]:
# 고객별 요약 ( RFM 만들기)
rfm=transactions.groupby('customer_id').agg({
    't_dat': 'max',
    'price': ['count','sum']
})

In [9]:
# RFM 컬럼 정리
rfm.columns=['latest_purchase','frequency','monetary']
rfm=rfm.reset_index()

In [10]:
# Recency 계산 : 마지막 구매 후 경과일
rfm['recency']=(latest_date-rfm['latest_purchase']).dt.days

In [11]:
# churn 정의 (90일 기준) : 이탈 여
rfm['churn']=(rfm['recency']>90).astype(int)

In [12]:
rfm.head()

,customer_id,latest_purchase,frequency,monetary,recency,churn
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2020-09-05,21,0.648983,17,0
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,2020-07-08,86,2.601932,76,0
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,2020-09-15,18,0.704780,7,0
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,2019-06-09,2,0.060983,471,1
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,2020-08-12,13,0.469695,41,0
